In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

### Helper functions that are used in the rest of the notebook

In [ ]:
def extract_nodes(df):
    """
    Extracts the nodes from a dataframe
    :param df: dataframe with source and target columns
    :return: set of nodes
    """
    return set(df['source'].unique()).union(set(df['target'].unique()))

def load_nodes(file_path):
    """
    Loads the nodes from a file
    :param file_path: path to the file
    :return: set of nodes
    """
    # much faster to do the operations on dataframes
    df = pd.read_csv(file_path, delimiter = ' ', names=['source', 'target', 'day'])
    nodes = extract_nodes(df)
    print(f"loaded {file_path} has {len(nodes)} nodes")
    return nodes

def largest_connected_component(G):
    """
    Returns the largest connected component of a graph
    :param G: graph
    :return: largest connected component
    """
    # not supported for directed networks
    subgraphs = sorted(nx.connected_components(G), key=len, reverse=True)
    return G.subgraph(subgraphs[0])

def build_network(df, directed = False):
    """
    Builds a network from a dataframe
    :param df: dataframe with source and target columns
    :param directed: whether the network is directed or not
    :return: networkx graph
    """
    if directed:
        G = nx.DiGraph()
    else:
        G = nx.Graph()

    # Check if exists a day column
    if "day" in df.columns:
        to_add = [(source, target, {"day": day}) for source, target, day in zip(df["source"], df["target"], df["day"])]
    else:
        to_add = zip(df["source"], df["target"])
    G.add_edges_from(to_add)
    return G

def extract_bidirectional_subnetwork(G):
    """
    creates an undirected sub network from a directed network

    that only contains the edges where both

    u -> v and v -> u

    are in the directed network. In the followers/following sense,

    This extracts the sub network where the users follow eachother.

    """
    sub_G = nx.Graph()

    for e in G.edges:
        if len(e) == 2:
            u, v = e
        else:
            u, v, _ = e
        if G.has_edge(v, u):
            sub_G.add_edge(u,v)

    return sub_G

def load_subnetwork(file_path, sub_nodes, directed = False):
    """
    Loads a network from a file and extracts a subnetwork by only keeping the nodes in sub_nodes
    :param file_path: path to the file with the network
    :param sub_nodes: set of nodes to keep
    :param directed: whether the network is directed or not
    :return: networkx graph
    """

    df = pd.read_csv(file_path, delimiter = ' ', names=['source', 'target', 'day'])
    if df['day'].isna().all():
        df.drop(columns=['day'], inplace=True)
    sub_df = df[((df['source'].isin(sub_nodes)) & (df['target'].isin(sub_nodes)))]

    return build_network(sub_df, directed)

def get_friendships(G):
    """
    Extracts the friendships from a social network defined as the subnetwork where both u -> v and v -> u are in the network
    :param G: social network
    :return: friendships network
    """

    G = extract_bidirectional_subnetwork(G)

    return G.to_undirected()

def get_lcc(G, directed = False):
    """
    Returns the largest connected component of a graph
    :param G: graph
    :param directed: whether the graph is directed or not
    :return: largest connected component
    """
    lcc = largest_connected_component(G.to_undirected())

    if directed:
        return lcc.to_directed()
    else:
        return lcc.to_undirected()

def get_subgraph(G, max_n_of_nodes=None, is_seed_node_most_connected=True, seed = None):
    """
    Returns a subgraph of G with k nodes generated by BFS from a seed node
    :param G: graph
    :param k: number of nodes in the subgraph
    :param is_seed_node_most_connected: whether the seed node is the most connected node or a random node
    :param seed: seed node, random if None
    :return: subgraph
    """
    if max_n_of_nodes is None:
        max_n_of_nodes = G.number_of_nodes() // 5

    if seed:
        seed_node = seed
    elif is_seed_node_most_connected:
        seed_node = max(G.degree(), key=lambda x: x[1])[0]
    else:
        # Choosing a random node does not guarantee getting a subgraph of the desired size
        seed_node = random.choice(list(G.nodes()))
    # Initialize a queue for BFS and a set for visited nodes
    queue = [seed_node]
    visited = set([seed_node])
    # Initialize the subgraph with the starting node
    subgraph = nx.Graph()
    subgraph.add_node(seed_node)
    # While the subgraph has fewer than n nodes and the queue is not empty
    while len(subgraph) < max_n_of_nodes and queue:
        # Get the next node from the queue
        curr_node = queue.pop(0)
        # Add its neighbors that have not been visited to the queue and the subgraph
        for neighbor in G.neighbors(curr_node):
            if neighbor not in visited:
                if len(subgraph) >= max_n_of_nodes:
                    break
                visited.add(neighbor)
                subgraph.add_node(neighbor)
                subgraph.add_edge(curr_node, neighbor, day=G[curr_node][neighbor]["day"])
                queue.append(neighbor)
    # print n of nodes in the subgraph 
    print(f"subgraph has {len(subgraph)} nodes")
    return subgraph


### Create the dataset of replies, mentions and retweets with real timestamps (not just days)

In [ ]:
activity_time_path = "../network-data/higgs-activity_time.txt"
activity_time = pd.read_csv(activity_time_path, delimiter = ' ', names=['source', 'target', 'time', 'type'])

In [ ]:
# create high resolution timestamped datasets
reply_timestamps_file_path = "../output/reply_timestamps.edgelist"
mention_timestamps_file_path = "../output/mention_timestamps.edgelist"
retweet_timestamps_file_path = "../output/retweet_timestamps.edgelist"

activity_time[activity_time['type'] == 'RE'][['source', 'target', 'time']]\
.to_csv(reply_timestamps_file_path, sep=' ', header=False, index=False)
replies = activity_time[activity_time['type'] == 'RE'].copy()
# replies[['source', 'target']] = replies[['target', 'source']]
replies[['source', 'target', 'time']].to_csv(reply_timestamps_file_path, sep=' ', header=False, index=False)
# activity_time[activity_time['type'] == 'MT'][['source', 'target', 'time']]\
# .to_csv(mention_timestamps_file_path, sep=' ', header=False, index=False)
mentions = activity_time[activity_time['type'] == 'MT'].copy()
#mentions[['source', 'target']] = mentions[['target', 'source']]
mentions[['source', 'target', 'time']].to_csv(retweet_timestamps_file_path, sep=' ', header=False, index=False)
# Swap source and target columns in the retweet dataset to effectively simulate the dataflow (as mentioned on the website)
retweets = activity_time[activity_time['type'] == 'RT'].copy()
retweets[['source', 'target']] = retweets[['target', 'source']]
retweets[['source', 'target', 'time']].to_csv(retweet_timestamps_file_path, sep=' ', header=False, index=False)

In [ ]:
social_file_path = "../network-data/higgs-social_network.edgelist"

### Create the intersection of nodes across all interaction mechanisms.

In [ ]:

reply_nodes = load_nodes(reply_timestamps_file_path)
mention_nodes = load_nodes(mention_timestamps_file_path)
reply_mention_nodes = reply_nodes.intersection(mention_nodes)
del reply_nodes, mention_nodes


retweet_nodes = load_nodes(retweet_timestamps_file_path)
reply_mention_retweet_nodes = reply_mention_nodes.intersection(retweet_nodes)
del reply_mention_nodes, retweet_nodes

print(f"intersected network has {len(reply_mention_retweet_nodes)} nodes")

### Create the social network and the friendship network

In [ ]:
# following/followers is a directed network
social_network = load_subnetwork(social_file_path, reply_mention_retweet_nodes, directed=True)
# Print information about the directionality of the network
lcc_social_network = get_lcc(social_network)
friends_network = get_friendships(social_network)
print(friends_network)
print(social_network)
print(lcc_social_network)
del social_network
lcc_friends_network = get_lcc(friends_network)
del friends_network

nx.write_edgelist(lcc_friends_network, '../output/higgs-friends-lcc.edgelist')
nx.write_edgelist(lcc_social_network, '../output/higgs-social-lcc.edgelist')

### Create the subnetworks for replies, mentions and retweets bby only keeping the nodes that are present in the social network or the friendship network.

In [ ]:
social_reply_network = load_subnetwork(reply_timestamps_file_path, lcc_social_network.nodes, directed=True)
social_mention_network = load_subnetwork(mention_timestamps_file_path, lcc_social_network.nodes, directed=True)
social_retweet_network = load_subnetwork(retweet_timestamps_file_path, lcc_social_network.nodes, directed=True)
print(social_reply_network)
print(social_mention_network)
print(social_retweet_network)
friends_reply_network = load_subnetwork(reply_timestamps_file_path, lcc_friends_network.nodes, directed=True)
friends_mention_network = load_subnetwork(mention_timestamps_file_path, lcc_friends_network.nodes, directed=True)
friends_retweet_network = load_subnetwork(retweet_timestamps_file_path, lcc_friends_network.nodes, directed=True)
print(friends_reply_network)
print(friends_mention_network)
print(friends_retweet_network)
paths = ['../output/higgs-social-reply.edgelist', '../output/higgs-social-mention.edgelist', '../output/higgs-social-retweet.edgelist', '../output/higgs-friends-reply.edgelist', '../output/higgs-friends-mention.edgelist', '../output/higgs-friends-retweet.edgelist']
networks = [social_reply_network, social_mention_network, social_retweet_network, friends_reply_network, friends_mention_network, friends_retweet_network]


for path, network in zip(paths, networks):
    nx.write_edgelist(network, path)
    with open(path, 'r') as f:
        data = f.readlines()

    # Remove the "day" key from each line of data
    data = [line.strip().replace("{'day': ", "").replace("}", "") for line in data]

    # Convert the data into a list of tuples with integer values
    data = [tuple(map(int, line.split())) for line in data]

    # Write the modified data to a new file
    with open(path, 'w') as f:
        for line in data:
            f.write(f"{line[0]} {line[1]} {line[2]}\n")



In [ ]:
friends_file_path = "../output/higgs-friends-lcc.edgelist"
friends_reply_path = "../output/higgs-friends-reply.edgelist"
friends_mention_path = "../output/higgs-friends-mention.edgelist"
friends_retweet_path = "../output/higgs-friends-retweet.edgelist"

### Temporal analysis of the reply network (Can be skipped now that we have a better way to do it presented in the next section)

In [ ]:
"""G_reply = nx.read_edgelist(friends_reply_path, data=(("day", int),), create_using=nx.DiGraph)
subgraph_reply = get_subgraph(G_reply, 5000, is_seed_node_most_connected=True)
print(subgraph_reply.number_of_nodes())
subgraph_reply_df = pd.DataFrame(subgraph_reply.edges(data=True), columns=['source', 'target', 'attribute'])
# Extract the 'day' attribute from the 'attribute' column and add it as a separate column
subgraph_reply_df['timestamp'] = [d['day'] for d in subgraph_reply_df['attribute']]
min_t = min(subgraph_reply_df['timestamp'])

# Hardcoded normalizing to ~1000 timesteps
subgraph_reply_df['day'] = ((subgraph_reply_df['timestamp'] - min_t)/ 600).astype(int)
# subgraph_reply_df['day'] = ((subgraph_reply_df['timestamp'] - min_t)).astype(int)
# Drop the 'attribute' column, which is no longer needed
subgraph_reply_df.drop('attribute', axis=1, inplace=True)

# there are days above 7, so we need to remove them
# subgraph_reply_df = subgraph_reply_df[subgraph_reply_df['day'] <= 7]
#unique_days = subgraph_reply_df['day'].unique()
#print(unique_days)

subgraph_reply_df.sort_values('day')
# nx.draw(subgraph_reply)"""

In [ ]:
"""#  First implementation of temporal spreading
# From the plot I am not sure if this is correct, the number of infected nodes is pretty low,
# but that could be because the huge majority of interactions happen at time 1, and i am considering a node as spreader from the next day is infected
# if we remove this condition I expect the number of infected nodes to be almost the same as the number of nodes in the subgraph at day 1

subgraph_reply_df = subgraph_reply_df.sort_values(by=['day'])
subgraph_reply_df.reset_index(drop=True, inplace=True)

iterations = 1
infected_nodes_time_dict = {0:[1 for _ in range(iterations)]}
seed_nodes = []

min_day = min(subgraph_reply_df['day'])
max_day = max(subgraph_reply_df['day'])
number_of_nodes = len(subgraph_reply.nodes())
for n in range(iterations):
    seed_node = random.choice(list(subgraph_reply.nodes()))
    seed_nodes.append(seed_node)
    infected_nodes = {seed_node}

    cur_timestamp = min_day
    nodes_infected_in_timestamp = set()

    # Iterate over all days
    for day in subgraph_reply_df['day']:
        print(f'Iteration {n+1}/{iterations} : {day}/{max_day} --- ', end='\r')
        # For each day get the edges that were created on that day
        edges = subgraph_reply_df[subgraph_reply_df['day'] == day]
        # Take the dataframe subset where either the source or the target is in the infected nodes
        susceptible_nodes_rows = edges[edges['source'].isin(infected_nodes) | edges['target'].isin(infected_nodes)]
        # Make a set of all the nodes that are in susceptible_nodes_rows
        susceptible_nodes = set(susceptible_nodes_rows['source']).union(set(susceptible_nodes_rows['target']))
        # Add the susceptible nodes to the infected nodes
        infected_nodes = infected_nodes.union(susceptible_nodes)
        # Add the infected nodes to the infected_nodes_time_dict
        if day in infected_nodes_time_dict:
            infected_nodes_time_dict[day].append(len(infected_nodes))
        else:
            infected_nodes_time_dict[day] = [len(infected_nodes)]

    # Take the average of the infected nodes for each timestamp
    infected_nodes_time_dict = {k: [np.sum(v)/(len(v)*number_of_nodes)] for k, v in infected_nodes_time_dict.items()}
    plt.clf()
    plt.plot(list(infected_nodes_time_dict.keys()), list(infected_nodes_time_dict.values()))"""

### Try to find correlations between different networks

In [ ]:
G_reply = nx.read_edgelist(friends_reply_path, data=(("day", int),), create_using=nx.DiGraph)
G_mention = nx.read_edgelist(friends_mention_path, data=(("day", int),), create_using=nx.DiGraph)
G_retweet = nx.read_edgelist(friends_retweet_path, data=(("day", int),), create_using=nx.DiGraph)
common_nodes = set(G_reply.nodes()).intersection(set(G_mention.nodes())).intersection(set(G_retweet.nodes()))

print(common_nodes)

In [ ]:
# Leave only the rows where the source and the target are unique pairs
# subgraph_reply_unique_df = subgraph_reply_df.drop_duplicates(subset=['source', 'target'])
# degrees = subgraph_reply_unique_df.groupby('source').count()
reply_degrees = G_reply.degree()
retweet_degrees = G_retweet.degree()
mention_degrees = G_mention.degree()

# Create a numpy array from reply_degrees where the index is the node id and the value is the degree
i = 0
node_index_dict = {}
for k, v in reply_degrees:
    node_index_dict[k] = i
    i += 1
for k,v in retweet_degrees:
    if k not in node_index_dict.keys():
        node_index_dict[k] = i
        i += 1
for k,v in mention_degrees:
    if k not in node_index_dict.keys():
        node_index_dict[k] = i
        i += 1

total_length = len(reply_degrees) + len(retweet_degrees) + len(mention_degrees)
reply_degrees_array = np.zeros(total_length)
for k, v in reply_degrees:
    reply_degrees_array[node_index_dict[k]] = v
retweet_degrees_array = np.zeros(total_length)
for k, v in retweet_degrees:
    retweet_degrees_array[node_index_dict[k]] = v
mention_degrees_array = np.zeros(total_length)
for k, v in mention_degrees:
    mention_degrees_array[node_index_dict[k]] = v

# Remove trailing zeros from the arrays
# reply_degrees_array = reply_degrees_array[~np.all(reply_degrees_array == 0)]
# retweet_degrees_array = retweet_degrees_array[~np.all(retweet_degrees_array == 0)]
# mention_degrees_array = mention_degrees_array[~np.all(mention_degrees_array == 0)]

# Calculate the correlation between the degrees of the different networks
print(f"Correlation between reply and retweet degrees: {np.corrcoef(reply_degrees_array, retweet_degrees_array)[0,1]}")
print(f"Correlation between reply and mention degrees: {np.corrcoef(reply_degrees_array, mention_degrees_array)[0,1]}")
print(f"Correlation between retweet and mention degrees: {np.corrcoef(retweet_degrees_array, mention_degrees_array)[0,1]}")

### Calculate autocorrelation of the number of infected nodes when starting from the node with maximum average degree

In [ ]:
# Get only the nodes that have non-zero degree in all the networks
common_nodes = set(G_reply.nodes()).intersection(set(G_mention.nodes())).intersection(set(G_retweet.nodes()))
# Construct a subgraph of each of the reply/mention/retweet graphs with only the common nodes but edges between nodes that are not in the common nodes
print(G_reply.number_of_nodes())
print(G_mention.number_of_nodes())
print(G_retweet.number_of_nodes())
print(len(common_nodes))
G_common_reply = G_reply.subgraph(common_nodes)
G_common_mention = G_mention.subgraph(common_nodes)
G_common_retweet = G_retweet.subgraph(common_nodes)
print(G_common_reply.number_of_nodes())
print(G_common_mention.number_of_nodes())
print(G_common_retweet.number_of_nodes())
# Create degrees_array with the degrees of the nodes in the common nodes subgraph
degrees_array = np.zeros(total_length)
# finding the average max degree in this way, has the risk of having a node with very low degree in one of the networks and very high degree in the other two, as it is the case
for node in G_common_reply.nodes():
    degrees_array[node_index_dict[node]] = np.sqrt(G_common_reply.out_degree(node)) + np.sqrt(G_common_mention.out_degree(node)) + np.sqrt(G_common_retweet.out_degree(node))
# Get the index of the node with the maximum average degree
# print("Degrees", degrees_array)
# Get index of the maximum degree
max_degree_index = np.argmax(degrees_array)
# Get the node with the maximum average degree
max_degree_node = list(node_index_dict.keys())[list(node_index_dict.values()).index(max_degree_index)]

In [ ]:
# Start spreading from the node with the maximal out_degree in each of the networks
max_degree_reply = max(G_reply.out_degree(), key=lambda x: x[1])[0]
max_degree_retweet = max(G_retweet.out_degree(), key=lambda x: x[1])[0]
max_degree_mention = max(G_mention.out_degree(), key=lambda x: x[1])[0]
print(G_reply.out_degree(max_degree_reply))
print(G_retweet.out_degree(max_degree_retweet))
print(G_mention.out_degree(max_degree_mention))
# Get the subgraph of the node with the maximum average degree
subgraph_reply = get_subgraph(G_reply, max_n_of_nodes=5000, seed=max_degree_reply)
subgraph_mention = get_subgraph(G_mention, max_n_of_nodes=5000, seed=max_degree_mention)
subgraph_retweet = get_subgraph(G_retweet, max_n_of_nodes=5000, seed=max_degree_retweet)
print(subgraph_retweet)
print(subgraph_mention)
print(subgraph_reply)

In [ ]:
# Functions for simulating spreading
def simulate_spreading(graph_df, seed_nodes, start_day, end_day, name="name_not_given", plot=True):

    infected_nodes_ratio_time_count = []
    infected_nodes_new = []
    
    for i, seed_node in enumerate(seed_nodes):
        
        infected_nodes_time = []
        infected_nodes = {seed_node}

        min_day = start_day
        max_day = end_day
        number_of_nodes = len(subgraph_reply.nodes())

        # Iterate over all days
        for day in np.arange(min_day, max_day+1):
            print(f'Graph: {name} Node: {i+1}/{len(seed_nodes)} Timestamp: {day}/{max_day} --- ', end='\r')
            # For each day get the edges that were created on that day
            edges = graph_df[graph_df['day'] == day]
            # Take the dataframe subset where either the source or the target is in the infected nodes
            susceptible_nodes_rows = edges[edges['source'].isin(infected_nodes) | edges['target'].isin(infected_nodes)]
            # Make a set of all the nodes that are in susceptible_nodes_rows
            susceptible_nodes = set(susceptible_nodes_rows['source']).union(set(susceptible_nodes_rows['target']))
            # Add the susceptible nodes to the infected nodes
            infected_nodes = infected_nodes.union(susceptible_nodes)
            # Add the infected nodes to the infected_nodes_time_dict
            infected_nodes_time.append(infected_nodes.copy())

        # Take the number of infected nodes per timestamp
        infected_nodes_ratio_time_count_it = [len(infected_nodes)/number_of_nodes for infected_nodes in infected_nodes_time]
        # Add legend to the plot

        last_infected_n = 0
        infected_nodes_new_it = []

        # add the difference between current infected and last infected
        for inf_nodes in infected_nodes_time:
            n_inf = len(inf_nodes) - last_infected_n
            infected_nodes_new_it.append(n_inf/number_of_nodes)
            last_infected_n = len(inf_nodes)
            
        infected_nodes_ratio_time_count.append(infected_nodes_ratio_time_count_it)
        infected_nodes_new.append(infected_nodes_new_it)
    
    # average over iterations
    infected_nodes_ratio_time_count = np.mean(infected_nodes_ratio_time_count, axis=0)
    infected_nodes_new = np.mean(infected_nodes_new, axis=0)

    if plot:
        plt.plot(np.arange(min_day, max_day+1), infected_nodes_new, label=f'{name}', alpha=0.8)
        plt.legend()
    return infected_nodes_ratio_time_count, infected_nodes_new

def build_df_from_graph(graph):
    graph_df = pd.DataFrame(graph.edges(data=True), columns=['source', 'target', 'attribute'])
    # Extract the 'day' attribute from the 'attribute' column and add it as a separate column
    graph_df['timestamp'] = [d['day'] for d in graph_df['attribute']]
    min_t = min(graph_df['timestamp'])

    # Hardcoded normalizing to ~1000 timesteps
    graph_df['day'] = ((graph_df['timestamp'] - min_t) / 600).astype(int)
    # subgraph_df['day'] = ((subgraph_df['timestamp'] - min_t)).astype(int)

    # Drop the 'attribute' column, which is no longer needed
    graph_df.drop('attribute', axis=1, inplace=True)

    graph_df.sort_values('day')

    # Simulate temporal spreading starting from the node with the highest average degree
    graph_df = graph_df.sort_values(by=['day'])
    graph_df.reset_index(drop=True, inplace=True)
    return graph_df

In [ ]:
# select random nodes
random_node = np.random.choice(G_common_reply.nodes(), 10)
# random_node = '1988'
print(random_node, [degrees_array[node_index_dict[node]] for node in random_node])

In [ ]:
# Simulate spreading for the node with the maximum average degree
print(subgraph_reply)
# Build dataframes for each graph
subgraph_reply_df = build_df_from_graph(subgraph_reply)
subgraph_mention_df = build_df_from_graph(subgraph_mention)
subgraph_retweet_df = build_df_from_graph(subgraph_retweet)

start_day_reply = min(subgraph_reply_df['day'])
start_day_mention = min(subgraph_mention_df['day'])
start_day_retweet = min(subgraph_retweet_df['day'])
end_day_reply = max(subgraph_reply_df['day'])
end_day_mention = max(subgraph_mention_df['day'])
end_day_retweet = max(subgraph_retweet_df['day'])

start_day = min(start_day_reply, start_day_mention, start_day_retweet)
end_day = max(end_day_reply, end_day_mention, end_day_retweet)
start_node = random_node

reply_infection_rates, reply_new_infected = simulate_spreading(subgraph_reply_df, start_node, start_day, end_day, name="reply")
mention_infection_rates, mention_new_infected = simulate_spreading(subgraph_mention_df, start_node, start_day, end_day, name="mention")
retweet_infection_rates, retweet_new_infected = simulate_spreading(subgraph_retweet_df, start_node, start_day, end_day, name="retweet")


In [ ]:
correlation_reply_mention = np.correlate(reply_new_infected, mention_new_infected, "full")
correlation_reply_retweet = np.correlate(reply_new_infected, retweet_new_infected, "full")
correlation_mention_retweet = np.correlate(mention_new_infected, retweet_new_infected, "full")
print(correlation_reply_mention)

correlation_reply_mention /= np.max(np.abs(correlation_reply_mention), axis=0)
correlation_reply_retweet /= np.max(np.abs(correlation_reply_retweet), axis=0)
correlation_mention_retweet /= np.max(np.abs(correlation_mention_retweet), axis=0)

plt.clf()
plt.plot(correlation_reply_mention, label="reply-mention", alpha=0.8)
plt.plot(correlation_reply_retweet, label="reply-retweet", alpha=0.8)
plt.plot(correlation_mention_retweet, label="mention-retweet", alpha=0.8)
plt.legend()
plt.title("Correlation between infection rates")

In [ ]:

correlation_reply_mention = np.corrcoef(reply_infection_rates, mention_infection_rates)[0][1]
corr_coefs = []
for i in range(len(reply_infection_rates)):
    corr_coefs.append(np.corrcoef(reply_infection_rates[:-i], mention_infection_rates[i:])[0][1])
correlation_reply_retweet = np.corrcoef(reply_infection_rates, retweet_infection_rates)[0][1]
correlation_mention_retweet = np.corrcoef(mention_infection_rates, retweet_infection_rates)[0][1]
print(correlation_reply_mention)
plt.clf()
plt.plot(corr_coefs, label="reply-mention")
plt.plot(correlation_reply_retweet, label="reply-retweet")
plt.plot(correlation_mention_retweet, label="mention-retweet")
plt.legend()
plt.title("Correlation between infection rates")